## CRM Agent with ADK

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Fcrm-agent-mcp.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [ ]:
%pip install google-adk -q
%pip install litellm -q

### Import Dependencies

In [ ]:
import os
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters

### Set Auth

In [ ]:
#os.environ["GOOGLE_API_KEY"] = "" # <--- REPLACE
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]="TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"]=""  # @param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"]="us-central1"